# SimFin Tutorial 05 - Stock Screener

[Original repository on GitHub](https://github.com/simfin/simfin-tutorials)

This tutorial was originally written by [Hvass Labs](https://github.com/Hvass-Labs)

----

"I am King Arthur and these are my knights of the round table. Go and tell your master that we have been charged by God with a quest for the Holy Grail and he can join us. Well I can ask him, but I don't think he'll be very keen, because he has already got one, you see."
&ndash; [Monty Python's Holy Grail](https://www.youtube.com/watch?v=M9DCAFUerzs)


## Introduction

A stock-screener is a very common tool used to search for stocks that meet certain criteria, e.g. low valuation ratios, high sales-growth, etc. This tutorial shows how to make a basic stock-screener using the signals calculated in the previous tutorial. It is assumed you are already familiar with the other previous tutorials on the basics of SimFin.

## Imports

In [1]:
%matplotlib inline
import pandas as pd
from datetime import datetime, timedelta

# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

In [2]:
# Version of the SimFin Python API.
sf.__version__

'0.3.0'

## Config

In [3]:
# SimFin data-directory.
sf.set_data_dir('~/simfin_data/')

In [4]:
# SimFin load API key or use free data.
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')

## Load Datasets

In these examples, we will use the following datasets:

In [5]:
%%time
# Data for USA.
market = 'us'

# TTM Income Statements.
df_income_ttm = sf.load_income(variant='ttm', market=market)

# Quarterly Income Statements.
df_income_qrt = sf.load_income(variant='quarterly', market=market)

# TTM Balance Sheets.
df_balance_ttm = sf.load_balance(variant='ttm', market=market)

# TTM Cash-Flow Statements.
df_cashflow_ttm = sf.load_cashflow(variant='ttm', market=market)

# Quarterly Cash-Flow Statements.
df_cashflow_qrt = sf.load_cashflow(variant='quarterly', market=market)

# Latest Share-Prices.
# Use refresh_days=0 to always download the latest share-prices.
df_prices_latest = sf.load_shareprices(variant='latest', market=market,
                                       refresh_days=30)

Dataset "us-income-ttm" on disk (16 days old), loading.
Dataset "us-income-quarterly" on disk (16 days old), loading.
Dataset "us-balance-ttm" on disk (20 days old), loading.
Dataset "us-cashflow-ttm" on disk (16 days old), loading.
Dataset "us-cashflow-quarterly" on disk (12 days old), loading.
Dataset "us-shareprices-latest" on disk (0 days old), loading.
CPU times: user 1.4 s, sys: 111 ms, total: 1.51 s
Wall time: 1.52 s


In [6]:
df_income_ttm.head()

SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2011-04-30      45846      USD         2011            Q2   2011-06-07   
       2011-07-31      45846      USD         2011            Q3   2011-09-07   
       2011-10-31      45846      USD         2011            Q4   2011-12-16   
       2012-01-31      45846      USD         2012            Q1   2012-03-05   
       2012-04-30      45846      USD         2012            Q2   2012-06-04   

                    Shares (Basic)  Shares (Diluted)       Revenue  \
Ticker Report Date                                                   
A      2011-04-30      484057500.0       495591000.0  6.156000e+09   
       2011-07-31      484407000.0       497338500.0  6.463000e+09   
       2011-10-31      485106000.0       496290000.0  6.615000e+09   
       2012-01-31      485455500.0       495241500.0  6.731000e+09   
       2012-04-30      485805000.0       494892000.0  6.787000e+09   

                    Cost of Revenue  Gross Profit  ...  \
Ticker Report Date                                 ...   
A      2011-04-30     -2.881000e+09  3.275000e+09  ...   
       2011-07-31     -3.021000e+09  3.442000e+09  ...   
       2011-10-31     -3.086000e+09  3.529000e+09  ...   
       2012-01-31     -3.144000e+09  3.587000e+09  ...   
       2012-04-30     -3.182000e+09  3.605000e+09  ...   

                    Non-Operating Income (Loss)  Interest Expense, Net  \
Ticker Report Date                                                       
A      2011-04-30                     1000000.0            -73000000.0   
       2011-07-31                    16000000.0            -69000000.0   
       2011-10-31                   -39000000.0            -72000000.0   
       2012-01-31                   -41000000.0            -76000000.0   
       2012-04-30                   -42000000.0            -82000000.0   

                    Pretax Income (Loss), Adj.  Abnormal Gains (Losses)  \
Ticker Report Date                                                        
A      2011-04-30                    796000000                      NaN   
       2011-07-31                    977000000                      NaN   
       2011-10-31                   1032000000                      NaN   
       2012-01-31                   1090000000                      NaN   
       2012-04-30                   1123000000                      NaN   

                    Pretax Income (Loss)  Income Tax (Expense) Benefit, Net  \
Ticker Report Date                                                            
A      2011-04-30              928000000                        -38000000.0   
       2011-07-31              982000000                         33000000.0   
       2011-10-31             1032000000                        -20000000.0   
       2012-01-31             1090000000                        -41000000.0   
       2012-04-30             1123000000                        -19000000.0   

                    Income (Loss) from Continuing Operations  \
Ticker Report Date                                             
A      2011-04-30                                  890000000   
       2011-07-31                                 1015000000   
       2011-10-31                                 1012000000   
       2012-01-31                                 1049000000   
       2012-04-30                                 1104000000   

                    Net Extraordinary Gains (Losses)  Net Income  \
Ticker Report Date                                                 
A      2011-04-30                                NaN   890000000   
       2011-07-31                                NaN  1015000000   
       2011-10-31                                NaN  1012000000   
       2012-01-31                                NaN  1049000000   
       2012-04-30                                NaN  1104000000   

                    Net Incom

In [7]:
df_prices_latest.head()

,,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume
Ticker,Date,,,,,,,,
A,2019-12-06,45846,82.24,81.82,82.42,82.210,82.21,NaN,1783441
AA,2019-12-06,367153,20.13,20.06,20.90,20.600,20.60,NaN,5272687
AAC,2019-11-04,939324,0.40,0.40,0.45,0.450,0.45,NaN,43300
AAL,2019-12-06,68568,27.50,27.47,27.96,27.725,27.73,NaN,6050312
AAME,2019-12-06,450021,1.70,1.70,2.00,1.960,1.96,NaN,11965


## Financial Signals

First we use the function `sf.fin_signals` from the simfin package to calculate financial signals for the stocks, such as the Current Ratio, Debt Ratio, Net Profit Margin, Return on Assets, etc. These are calculated using data from the financial reports: Income Statements, Balance Sheets and Cash-Flow Statements.

Note that we are only using the latest share-prices, because we are only interested in the most recent signals, and we are not interested in the signals from several years ago.

In [8]:
%%time
df_fin_signals = \
    sf.fin_signals(df_prices=df_prices_latest,
                   df_income_ttm=df_income_ttm,
                   df_balance_ttm=df_balance_ttm,
                   fill_method='ffill')

CPU times: user 15 s, sys: 56.1 ms, total: 15 s
Wall time: 15 s


In [9]:
df_fin_signals.dropna().head()

,,Current Ratio,Debt Ratio,Gross Profit Margin,Net Profit Margin,Return on Assets,Return on Equity
Ticker,Date,,,,,,
A,2019-12-06,3.409660,0.210631,0.547121,0.222554,0.127163,0.241932
AA,2019-12-06,1.416238,0.113063,0.247855,0.016937,0.013011,0.033392
AAL,2019-12-06,0.489490,0.408137,0.264137,0.033458,0.028638,-1.734177
AAOI,2019-12-06,3.118895,0.228300,0.291388,-0.057816,-0.030991,-0.042937
AAP,2019-12-06,1.363659,0.123951,0.439722,0.045177,0.049414,0.119358


We then pass the argument `func=sf.avg_ttm_2y` to `sf.fin_signals` so as to calculate 2-year averages of the financial signals. We then get another DataFrame with the 2-year average Current Ratio, Net Profit Margin, Return on Assets, etc.

In [10]:
%%time
df_fin_signals_2y = \
    sf.fin_signals(df_prices=df_prices_latest,
                   df_income_ttm=df_income_ttm,
                   df_balance_ttm=df_balance_ttm,
                   fill_method='ffill',
                   func=sf.avg_ttm_2y)

CPU times: user 21.2 s, sys: 12.2 ms, total: 21.2 s
Wall time: 21.2 s


In [11]:
df_fin_signals_2y.dropna().head()

,,Current Ratio,Debt Ratio,Gross Profit Margin,Net Profit Margin,Return on Assets,Return on Equity
Ticker,Date,,,,,,
A,2019-12-06,3.362339,0.224649,0.544747,0.136383,0.078364,0.148052
AAL,2019-12-06,0.519986,0.432832,0.275318,0.035501,0.029637,-0.647169
AAOI,2019-12-06,3.290255,0.182644,0.360077,0.051187,0.061052,0.080428
AAP,2019-12-06,1.498598,0.141038,0.437761,0.050989,0.056415,0.145199
AAPL,2019-12-06,1.336526,0.316104,0.380807,0.218261,0.154850,0.479904


## Growth Signals

Now we use the function `sf.growth_signals` to calculate growth signals for the stocks, such as Earnings Growth, FCF Growth, Sales Growth, etc. These are also calculated using data from the financial reports: Income Statements, Balance Sheets and Cash-Flow Statements.

In [12]:
%%time
df_growth_signals = \
    sf.growth_signals(df_prices=df_prices_latest,
                      df_income_ttm=df_income_ttm,
                      df_income_qrt=df_income_qrt,
                      df_cashflow_ttm=df_cashflow_ttm,
                      df_cashflow_qrt=df_cashflow_qrt,
                      fill_method='ffill')

CPU times: user 14.8 s, sys: 40 ms, total: 14.9 s
Wall time: 14.9 s


In [13]:
df_growth_signals.dropna().head()

,,Earnings Growth,Earnings Growth QOQ,Earnings Growth YOY,FCF Growth,FCF Growth QOQ,FCF Growth YOY,Sales Growth,Sales Growth QOQ,Sales Growth YOY
Ticker,Date,,,,,,,,,
A,2019-12-06,3.713080,-0.638889,-0.112195,0.076364,0.224138,-0.164706,0.063347,-0.035826,0.026534
AA,2019-12-06,0.046083,-2.048780,-1.219388,-0.940171,0.878641,0.264706,0.150275,-0.013569,0.053560
AAL,2019-12-06,-0.077723,2.578378,0.169611,4.191781,-1.147518,-1.282609,0.034902,0.130008,0.027227
AAN,2019-12-06,-0.358158,-0.066800,-0.089639,-0.082894,0.564822,0.146337,0.058293,-0.004476,0.011266
AAOI,2019-12-06,-1.261895,0.222456,-5.940566,-60.530462,-0.587227,-0.067186,-0.274372,-0.091334,-0.191910


We then pass the argument `func=sf.avg_ttm_2y` to `sf.growth_signals` so as to calculate 2-year averages of the growth signals. We then get another DataFrame with the 2-year average Earnings Growth, FCF Growth, Sales Growth, etc.

In [14]:
%%time
df_growth_signals_2y = \
    sf.growth_signals(df_prices=df_prices_latest,
                      df_income_ttm=df_income_ttm,
                      df_income_qrt=df_income_qrt,
                      df_cashflow_ttm=df_cashflow_ttm,
                      df_cashflow_qrt=df_cashflow_qrt,
                      fill_method='ffill',
                      func=sf.avg_ttm_2y)

CPU times: user 23.2 s, sys: 32.1 ms, total: 23.2 s
Wall time: 23.2 s


In [15]:
df_growth_signals_2y.dropna().head()

,,Earnings Growth,Earnings Growth QOQ,Earnings Growth YOY,FCF Growth,FCF Growth QOQ,FCF Growth YOY,Sales Growth,Sales Growth QOQ,Sales Growth YOY
Ticker,Date,,,,,,,,,
A,2019-12-06,1.560148,-1.139757,0.068902,0.172797,0.434650,0.013441,0.077465,-0.019977,0.060454
AAL,2019-12-06,-0.142836,2.310695,-0.062766,1.806873,-0.897175,-0.765026,0.046774,0.124710,0.037837
AAN,2019-12-06,0.462982,0.034378,0.317814,0.450612,0.468240,0.311686,0.093859,0.011348,0.073692
AAOI,2019-12-06,-0.593424,-0.203328,-3.416718,-30.736077,-1.901287,-3.713263,-0.064059,-0.137183,-0.256959
AAP,2019-12-06,0.128676,-0.131118,0.206472,0.510563,0.948150,0.215537,0.009474,-0.200179,0.015101


## Valuation Signals

Now we use the function `sf.val_signals` to calculate valuation signals for the stocks, such as P/E, P/Sales, etc. These are calculated from the share-prices and data from the financial reports.

In [16]:
%%time
df_val_signals = \
    sf.val_signals(df_prices=df_prices_latest,
                   df_income_ttm=df_income_ttm,
                   df_balance_ttm=df_balance_ttm,
                   df_cashflow_ttm=df_cashflow_ttm,
                   fill_method='ffill')

CPU times: user 4.98 s, sys: 44.1 ms, total: 5.02 s
Wall time: 4.45 s


In [17]:
df_val_signals.dropna().head()

,,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales
Ticker,Date,,,,,,,,,,
A,2019-12-06,0.007483,0.042000,0.033390,2.659493e+10,5.189256,23.809252,29.949251,-312.881588,-33.274864,5.298851
AAL,2019-12-06,0.014408,0.119302,0.030004,1.263179e+10,-574.172148,8.382075,33.329254,-0.242271,-0.232755,0.280451
AAN,2019-12-06,0.002247,0.049222,0.063362,4.068722e+09,2.367657,20.315981,15.782230,8.846648,-15.988344,1.033396
AAP,2019-12-06,0.001548,0.038731,0.055512,1.127280e+10,3.070519,25.819285,18.014234,-6.898170,-2.731700,1.166433
AAPL,2019-12-06,0.011219,0.043906,0.046798,1.258507e+12,13.908001,22.775938,21.368297,-14.769652,-9.814950,4.837175


We then pass the argument `func=sf.avg_ttm_2y` to `sf.val_signals` so as to calculate the valuation signals using 2-year averages of the financial data. We then get another DataFrame with e.g. P/E and P/Sales ratios calculated from the 2-year average Earnings and Sales.

In [18]:
%%time
df_val_signals_2y = \
    sf.val_signals(df_prices=df_prices_latest,
                   df_income_ttm=df_income_ttm,
                   df_balance_ttm=df_balance_ttm,
                   df_cashflow_ttm=df_cashflow_ttm,
                   fill_method='ffill',
                   func=sf.avg_ttm_2y)

CPU times: user 14.9 s, sys: 60 ms, total: 14.9 s
Wall time: 14.3 s


In [19]:
df_val_signals_2y.dropna().head()

,,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales
Ticker,Date,,,,,,,,,,
A,2019-12-06,0.007144,0.025456,0.032205,2.659493e+10,5.459851,39.283508,31.050712,194.834689,-48.675241,5.461533
AAL,2019-12-06,0.014725,0.124329,0.017891,1.263179e+10,-28.354180,8.043163,55.892864,-0.261924,-0.250454,0.285261
AAN,2019-12-06,0.001898,0.062956,0.066226,4.068722e+09,2.389852,15.884170,15.099821,8.369662,-18.188544,1.062662
AAP,2019-12-06,0.001567,0.043098,0.056967,1.127280e+10,3.076113,23.203113,17.554057,-24.139695,-3.871134,1.181384
AAPL,2019-12-06,0.011057,0.045604,0.048874,1.258507e+12,12.735672,21.927696,20.460704,-11.847673,-8.358481,4.787301


## Combine Signals

We now combine all the basic signals into a single DataFrame:

In [20]:
# Combine the DataFrames.
dfs = [df_fin_signals, df_growth_signals, df_val_signals]
df_signals = pd.concat(dfs, axis=1)

# Show the result.
df_signals.head()

,,Current Ratio,Debt Ratio,Gross Profit Margin,Net Profit Margin,Return on Assets,Return on Equity,Earnings Growth,Earnings Growth QOQ,Earnings Growth YOY,FCF Growth,...,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales
Ticker,Date,,,,,,,,,,,,,,,,,,,,,
A,2019-12-06,3.409660,0.210631,0.547121,0.222554,0.127163,0.241932,3.713080,-0.638889,-0.112195,0.076364,...,0.007483,0.042000,0.033390,2.659493e+10,5.189256,23.809252,29.949251,-312.881588,-33.274864,5.298851
AA,2019-12-06,1.416238,0.113063,0.247855,0.016937,0.013011,0.033392,0.046083,-2.048780,-1.219388,-0.940171,...,NaN,0.058304,0.012585,3.893400e+09,0.526562,17.151542,79.457143,-0.882857,-0.650363,0.290487
AAC,2019-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAL,2019-12-06,0.489490,0.408137,0.264137,0.033458,0.028638,-1.734177,-0.077723,2.578378,0.169611,4.191781,...,0.014408,0.119302,0.030004,1.263179e+10,-574.172148,8.382075,33.329254,-0.242271,-0.232755,0.280451
AAME,2019-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Then we combine all the signals for multi-year averages into another DataFrame:

In [21]:
# Combine the DataFrames.
dfs = [df_fin_signals_2y, df_growth_signals_2y, df_val_signals_2y]
df_signals_2y = pd.concat(dfs, axis=1)

# Show the result.
df_signals_2y.head()

,,Current Ratio,Debt Ratio,Gross Profit Margin,Net Profit Margin,Return on Assets,Return on Equity,Earnings Growth,Earnings Growth QOQ,Earnings Growth YOY,FCF Growth,...,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales
Ticker,Date,,,,,,,,,,,,,,,,,,,,,
A,2019-12-06,3.362339,0.224649,0.544747,0.136383,0.078364,0.148052,1.560148,-1.139757,0.068902,0.172797,...,0.007144,0.025456,0.032205,2.659493e+10,5.459851,39.283508,31.050712,194.834689,-48.675241,5.461533
AA,2019-12-06,1.359718,0.096768,0.234638,0.017780,NaN,NaN,NaN,-2.391647,-0.325694,NaN,...,NaN,0.057020,0.111471,3.893400e+09,0.548675,17.537838,8.970968,-0.719601,-0.558484,0.310788
AAC,2019-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAL,2019-12-06,0.519986,0.432832,0.275318,0.035501,0.029637,-0.647169,-0.142836,2.310695,-0.062766,1.806873,...,0.014725,0.124329,0.017891,1.263179e+10,-28.354180,8.043163,55.892864,-0.261924,-0.250454,0.285261
AAME,2019-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Screener for Net-Net Stocks

This is an old investment strategy used by Ben Graham who was the teacher of Warren Buffett, who also used the strategy when he started investing. The idea is to buy stocks that are cheaper than a conservative estimate of their liquidation value. In normal market conditions, few companies have stocks that trade at such low prices, and there may be very good reasons why the stocks are so cheap. But during market panics, it is sometimes possible to buy decent stocks at such low prices.

The Net-Net formula is:

$$
    NetNet = Cash\ \&\ Equiv + 0.75 \cdot Receivables \\
    + 0.5 \cdot Inventories - Total\ Liabilities
$$

This means P/NetNet ratios between 0 and 1 indicate the stocks are trading at a discount to their estimated liquidation values. The lower the P/NetNet ratio, the cheaper the stock is.

We create the stock-screener for Net-Net stocks, by making a boolean mask for the rows in the DataFrame with signals that meet the criteria: P/NetNet > 0 and P/NetNet < 1

In [22]:
mask = (df_signals[P_NETNET] > 0) & (df_signals[P_NETNET] < 1)

Rows that satisfy the screener-condition have a value of `True` and rows that do not meet the condition have a value of `False`.

In [23]:
mask.head()

Ticker  Date      
A       2019-12-06    False
AA      2019-12-06    False
AAC     2019-11-04    False
AAL     2019-12-06    False
AAME    2019-12-06    False
Name: P/NetNet, dtype: bool

We can then use the boolean mask to select matching rows in the signal DataFrame, and show the P/NetNet signal:

In [24]:
df_signals.loc[mask, P_NETNET]

Ticker  Date      
ADIL    2019-12-06    0.311631
AEHR    2019-12-06    0.007679
ALPN    2019-12-06    0.265546
AMDA    2019-12-06    0.183206
AVGR    2019-12-06    0.319653
AXGN    2019-12-06    0.005119
CGA     2019-12-06    0.048210
CLBS    2019-12-06    0.701183
CLRB    2019-12-06    0.703117
CUO     2019-12-06    0.551325
CYCC    2019-12-06    0.328075
CYIG    2019-12-06    0.087683
EVLO    2019-12-06    0.731361
FPRX    2019-12-06    0.607481
IFON    2018-07-10    0.036616
KKR     2019-12-06    0.706922
MN      2019-12-06    0.291478
NHTC    2019-12-06    0.986474
NLNK    2019-12-06    0.538685
NSPR    2019-12-06    0.057442
PESI    2019-12-06    0.757295
RDC     2019-04-11    0.362291
RKDA    2019-12-06    0.805419
SOHU    2019-12-06    0.544774
SPRT    2019-12-06    0.842042
SRRA    2019-12-06    0.554963
SVT     2019-12-06    0.003531
TOCA    2019-12-06    0.440308
TROV    2019-12-06    0.639637
UMRX    2019-12-06    0.755950
WINH    2019-12-05    0.511737
WSTL    2019-12-06  

Note that some of the dates are not recent, so we can remove all rows with dates that are older than e.g. 30 days, by creating another boolean mask.

In [25]:
# Oldest date that is allowed for a row.
date_limit = datetime.now() - timedelta(days=30)

# Boolean mask for the tickers that satisfy this condition.
mask_date_limit = (df_prices_latest.reset_index(DATE)[DATE] > date_limit)

# Show the result.
mask_date_limit.head()

Ticker
A        True
AA       True
AAC     False
AAL      True
AAME     True
Name: Date, dtype: bool

We can then combine the screener-mask and the date-mask:

In [26]:
mask &= mask_date_limit

And then we can show the recent stock-prices that are trading at Net-Net discounts:

In [27]:
df_signals.loc[mask, P_NETNET]

Ticker  Date      
ADIL    2019-12-06    0.311631
AEHR    2019-12-06    0.007679
ALPN    2019-12-06    0.265546
AMDA    2019-12-06    0.183206
AVGR    2019-12-06    0.319653
AXGN    2019-12-06    0.005119
CGA     2019-12-06    0.048210
CLBS    2019-12-06    0.701183
CLRB    2019-12-06    0.703117
CUO     2019-12-06    0.551325
CYCC    2019-12-06    0.328075
CYIG    2019-12-06    0.087683
EVLO    2019-12-06    0.731361
FPRX    2019-12-06    0.607481
KKR     2019-12-06    0.706922
MN      2019-12-06    0.291478
NHTC    2019-12-06    0.986474
NLNK    2019-12-06    0.538685
NSPR    2019-12-06    0.057442
PESI    2019-12-06    0.757295
RKDA    2019-12-06    0.805419
SOHU    2019-12-06    0.544774
SPRT    2019-12-06    0.842042
SRRA    2019-12-06    0.554963
SVT     2019-12-06    0.003531
TOCA    2019-12-06    0.440308
TROV    2019-12-06    0.639637
UMRX    2019-12-06    0.755950
WINH    2019-12-05    0.511737
WSTL    2019-12-06    0.582004
Name: P/NetNet, dtype: float64

## Screener for Many Criteria

It is very easy to combine many criteria in the stock-screener. Let use start with the condition that the Market Capitalization must be more then USD 1 billion:

In [28]:
mask = (df_signals[MARKET_CAP] > 1e9)

Then let us add criteria for the Current Ratio and Debt Ratio calculated from the latest financial reports, as well as the quarterly sales-growth year-over-year.

We combine all these criteria simply by generating the corresponding boolean masks, and doing the logical-and with the previous mask, thereby accumulating multiple criteria.

In [29]:
mask &= (df_signals[CURRENT_RATIO] > 2)
mask &= (df_signals[DEBT_RATIO] < 0.5)
mask &= (df_signals[SALES_GROWTH_YOY] > 0.1)

We can also create screener-criteria using the 2-year average signals, e.g. the P/E and P/FCF ratios which use 2-year average Earnings and FCF. We can combine screener-criteria from `df_signals` and `df_signals_2y` because their indices are compatible.

In [30]:
mask &= (df_signals_2y[PE] < 20)
mask &= (df_signals_2y[PFCF] < 20)
mask &= (df_signals_2y[ROA] > 0.03)
mask &= (df_signals_2y[ROE] > 0.15)
mask &= (df_signals_2y[NET_PROFIT_MARGIN] > 0.0)
mask &= (df_signals_2y[SALES_GROWTH] > 0.1)

Finally we can ensure that we only get the stocks with recent share-prices:

In [31]:
mask &= mask_date_limit

These are the stocks and signals matching all these criteria:

In [32]:
df_signals[mask]

,,Current Ratio,Debt Ratio,Gross Profit Margin,Net Profit Margin,Return on Assets,Return on Equity,Earnings Growth,Earnings Growth QOQ,Earnings Growth YOY,FCF Growth,...,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales
Ticker,Date,,,,,,,,,,,,,,,,,,,,,
CORT,2019-12-06,6.657296,0.297553,0.979244,0.300143,0.341938,0.394883,-0.415979,0.240097,-0.776182,0.906388,...,NaN,0.044755,0.068469,1.684950e+09,6.107504,22.343859,14.605133,8.382504,9.034344,6.706350
EXEL,2019-12-06,8.639725,0.013915,0.967346,0.704107,0.704089,0.827823,0.984815,0.043114,-0.096601,1.653536,...,NaN,0.117800,0.099365,5.445984e+09,3.658382,8.488976,10.063891,6.783897,7.201061,5.977149
JAZZ,2019-12-06,3.153672,0.302764,1.000000,0.248815,0.091980,0.174551,0.087140,-0.465724,0.852558,0.134549,...,NaN,0.053829,0.072006,9.034371e+09,3.309390,18.577468,13.887730,-7.258284,-6.242950,4.622356
PRGO,2019-12-06,3.183860,0.360024,0.364800,0.120889,0.108734,0.227802,0.025325,-0.059288,0.054924,0.245239,...,0.006873,0.090506,0.097561,4.946634e+09,2.002767,11.048993,10.249967,-7.364350,-3.706316,1.335701


We can also show a sub-set of all the signals and sort e.g. by the P/FCF ratios:

In [33]:
columns = [PFCF, PE, ROA, ROE, CURRENT_RATIO, DEBT_RATIO]
df_signals.loc[mask, columns].sort_values(by=PFCF, ascending=True)

,,P/FCF,P/E,Return on Assets,Return on Equity,Current Ratio,Debt Ratio
Ticker,Date,,,,,,
EXEL,2019-12-06,10.063891,8.488976,0.704089,0.827823,8.639725,0.013915
PRGO,2019-12-06,10.249967,11.048993,0.108734,0.227802,3.183860,0.360024
JAZZ,2019-12-06,13.887730,18.577468,0.091980,0.174551,3.153672,0.302764
CORT,2019-12-06,14.605133,22.343859,0.341938,0.394883,6.657296,0.297553


## Handling NaN Signal-Values

The signals are calculated from various financial data using mathematical formulas. If any data-item in the formula is NaN (Not-a-Number) then the result of the entire formula is also NaN, and then the screener-condition automatically evaluates to False, so the company is excluded from the results of the stock-screener.

For example, the Debt Ratio (`DEBT_RATIO`) is calculated from Short Term Debt (`ST_DEBT`), Long Term Debt (`LT_DEBT`) and Total Assets (`TOTAL_ASSETS`). If just one of these numbers is NaN, then the resulting Debt Ratio is also NaN and the screener-condition for this signal will always evaluate to False, so the company is excluded from the screener's results.

You might think that a solution would simply be to use `fillna(0)` on all the data-items before calculating the signals. This may work for some formulas and for some uses of the signals, but it is not a generally valid solution, as it may severely distort the signals.

Consider for example the ticker AMZN, where all data for Short Term Debt is missing in all the reports, while the Long Term Debt is only missing in some reports. If you look at the data, it seems most likely that this is a data-error, and the Long Term Debt should actually be several billions of dollars. If we were to replace these missing values with zeros, then we would get very misleading Debt Ratios.

In this example, AMZN had actually not reported these numbers in some of their quarterly reports. That is why the values are missing in the data.

In [34]:
columns = [ST_DEBT, LT_DEBT, TOTAL_ASSETS]
df_balance_ttm.loc['AMZN', columns]['2010':'2013']

,Short Term Debt,Long Term Debt,Total Assets
Report Date,,,
2010-03-31,NaN,1.310000e+08,12042000000
2010-06-30,NaN,1.320000e+08,12397000000
2010-09-30,NaN,1.640000e+08,14162000000
2010-12-31,NaN,NaN,18797000000
2011-03-31,NaN,NaN,16882000000
2011-06-30,NaN,NaN,17941000000
2011-09-30,NaN,NaN,19054000000
2011-12-31,NaN,2.550000e+08,25278000000
2012-03-31,NaN,NaN,20339000000


A simple solution is to ignore signals that are NaN. For example, we could have the following criteria:

In [35]:
# Start the screener with a market-cap condition.
mask = (df_signals[MARKET_CAP] > 1e9)

# Ensure share-prices are recent.
mask &= mask_date_limit

# Screener criteria where NaN signals are ignored.
mask &= ((df_signals[CURRENT_RATIO] > 2) | (df_signals[CURRENT_RATIO].isnull()))
mask &= ((df_signals[DEBT_RATIO] < 0.5) | (df_signals[DEBT_RATIO].isnull()))
mask &= ((df_signals[PE] < 20) | (df_signals[PE].isnull()))
mask &= ((df_signals[PFCF] < 20) | (df_signals[PFCF].isnull()))
mask &= ((df_signals[ROA] > 0.03) | (df_signals[ROA].isnull()))
mask &= ((df_signals[ROE] > 0.15) | (df_signals[ROE].isnull()))
mask &= ((df_signals[NET_PROFIT_MARGIN] > 0.0) | (df_signals[NET_PROFIT_MARGIN].isnull()))
mask &= ((df_signals[SALES_GROWTH] > 0.1) | (df_signals[SALES_GROWTH].isnull()))

The following shows the stocks whose signals match these criteria. You can see that some of the signals are NaN, but the stocks are still included in the results, because the screener just ignores NaN values:

In [36]:
columns = [PFCF, PE, ROA, ROE, CURRENT_RATIO, DEBT_RATIO]
df_signals.loc[mask, columns].sort_values(by=PE, ascending=True)

,,P/FCF,P/E,Return on Assets,Return on Equity,Current Ratio,Debt Ratio
Ticker,Date,,,,,,
HFC,2019-12-06,12.783679,6.140207,0.145555,0.270254,2.503984,NaN
NUE,2019-12-06,9.727368,7.198040,0.155596,0.269057,3.368494,0.238033
EXEL,2019-12-06,10.063891,8.488976,0.704089,0.827823,8.639725,0.013915
PRGO,2019-12-06,10.249967,11.048993,0.108734,0.227802,3.183860,0.360024
PATK,2019-12-06,6.979344,11.831919,0.086688,0.255609,2.007255,0.435674
IRBT,2019-12-06,-37.877778,14.407794,0.130783,0.179615,2.406644,NaN
UFS,2019-12-06,13.380670,16.632917,NaN,NaN,2.132597,0.222933
HDS,2019-12-06,12.380920,16.785670,0.090317,0.258706,2.054779,0.435870
ALXN,2019-12-06,17.186309,17.480872,0.106208,0.160343,3.978372,0.170996


## License (MIT)

This is published under the
[MIT License](https://github.com/simfin/simfin-tutorials/blob/master/LICENSE.txt)
which allows very broad use for both academic and commercial purposes.

You are very welcome to modify and use this source-code in your own project. Please keep a link to the [original repository](https://github.com/simfin/simfin-tutorials).
